<a href="https://colab.research.google.com/github/schauppi/Udemy_TensorFlow_Developer/blob/main/Abschnitt_6_Transfer_Learning/05_transfer_learning_in_tensorflow_part_2_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer Leaning in TensorFlow Part 2: Fine-tuning


In [1]:
!nvidia-smi

Sat Jul 17 11:10:23 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Creating helper functions

In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2021-07-17 11:12:49--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2021-07-17 11:12:49 (91.1 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [4]:
#Import helper functions we´re going to use in this notebook
from helper_functions import create_tensorboard_callback, plot_loss_curves, unzip_data, walk_through_dir

## Let´s get some data

This time weé going to see how we can use the pretrained models with tf.keras.applications ans apply them to out own problem.

In [9]:
#get 10% of training data from 10_food_classes_10_percent
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

--2021-07-17 11:20:24--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.12.240, 172.217.15.112, 172.217.164.144, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.12.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip’

10_food_classes_10_ 100%[===================>] 160.74M   141MB/s    in 1.1s    

2021-07-17 11:20:26 (141 MB/s) - ‘10_food_classes_10_percent.zip’ saved [168546183/168546183]



In [10]:
#Unzip data using the helper function
unzip_data("10_food_classes_10_percent.zip")

In [13]:
#Check out how many images and subdirectories are in our dataset 
walk_through_dir("10_food_classes_10_percent")

There are 2 directories and 0 images in '10_food_classes_10_percent'.
There are 10 directories and 0 images in '10_food_classes_10_percent/test'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/hamburger'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/grilled_salmon'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/pizza'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/chicken_curry'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/sushi'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/ice_cream'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/fried_rice'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/chicken_wings'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/steak'.
There are 0 directories and 250 images in '10_food_classes_10_percent

In [14]:
#Create training an test directory paths
train_dir = "10_food_classes_10_percent/train"
test_dir = "10_food_classes_10_percent/test"

In [19]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory

IMG_SIZE = (224,224)
BATCH_SIZE=32
train_data_10_percent = image_dataset_from_directory(train_dir,
                                                     image_size=IMG_SIZE,
                                                     label_mode="categorical",
                                                     batch_size=BATCH_SIZE)

test_data_10_percent = image_dataset_from_directory(test_dir,
                                                    image_size=IMG_SIZE,
                                                    label_mode="categorical",
                                                    batch_size=BATCH_SIZE)

Found 750 files belonging to 10 classes.
Found 2500 files belonging to 10 classes.


In [20]:
train_data_10_percent

<BatchDataset shapes: ((None, 224, 224, 3), (None, 10)), types: (tf.float32, tf.float32)>

In [21]:
#Check out the class names of our dataset
train_data_10_percent.class_names

['chicken_curry',
 'chicken_wings',
 'fried_rice',
 'grilled_salmon',
 'hamburger',
 'ice_cream',
 'pizza',
 'ramen',
 'steak',
 'sushi']

In [26]:
#see an example of a batch of data
for images, labels in train_data_10_percent.take(1):
  print(images, labels)

tf.Tensor(
[[[[2.47000000e+02 2.34000000e+02 2.18000000e+02]
   [2.47000000e+02 2.34000000e+02 2.18000000e+02]
   [2.47000000e+02 2.34000000e+02 2.18000000e+02]
   ...
   [2.47000000e+02 2.34000000e+02 2.18000000e+02]
   [2.47000000e+02 2.34000000e+02 2.18000000e+02]
   [2.47000000e+02 2.34000000e+02 2.18000000e+02]]

  [[2.47000000e+02 2.34000000e+02 2.18000000e+02]
   [2.47000000e+02 2.34000000e+02 2.18000000e+02]
   [2.47000000e+02 2.34000000e+02 2.18000000e+02]
   ...
   [2.47000000e+02 2.34000000e+02 2.18000000e+02]
   [2.47000000e+02 2.34000000e+02 2.18000000e+02]
   [2.47000000e+02 2.34000000e+02 2.18000000e+02]]

  [[2.47000000e+02 2.34000000e+02 2.18000000e+02]
   [2.47000000e+02 2.34000000e+02 2.18000000e+02]
   [2.47000000e+02 2.34000000e+02 2.18000000e+02]
   ...
   [2.47000000e+02 2.34000000e+02 2.18000000e+02]
   [2.47000000e+02 2.34000000e+02 2.18000000e+02]
   [2.47000000e+02 2.34000000e+02 2.18000000e+02]]

  ...

  [[2.47000000e+02 2.34000000e+02 2.18000000e+02]
   [2